# Fonction of metoo project
#### this file contain all the main fonction use in my research project on the #metoo campaign and the french equivalent #balancetonporc.

In [1]:
import pandas as pd
import json
import csv

import numpy as np
import pylab
import nltk
import operator 
import json

import datetime
import time
import  tweepy, sys, locale, threading 
from time import localtime, strftime, sleep



import nltk
import stop_words
from nltk.corpus import stopwords
from nltk import bigrams 
from nltk import trigrams
from collections import Counter
from collections import defaultdict

import wordcloud

import matplotlib.pyplot as plt
import numpy as np

from labMTsimple.storyLab import *



## behavior function

In [2]:
def has_picture(df):
    df_url_i=df[df['url_image'].apply(lambda x: '/' in x)]
    per=round(len(df_url_i)/len(df)*100,3)
    print(per,'% of the tweet have a picture')
    return(df_url_i)
#################
def has_url(df):
    df_url=df[df['url'].apply(lambda x: '/' in x)]
    per=round(len(df_url)/len(df)*100,3)
    print(per,'% of the tweet have a url')
    return(df_url)
#################
def has_mention(df):
    df_men=df[df['mention'].apply(lambda x: '@' in x)]
    per=round(len(df_men)/len(df)*100,3)
    print(per,'% of the tweet have a mention')
    return(df_men)
#################
def mean_nb_word(df):
    s=(df['nb_mots']).sum()
    m=s/len(df)
    print('the average number of word =',round(m,3))
    return(m)
#############
def behavior(df):
    i1=mean_nb_word(df)
    i2=has_mention(df)
    i3=has_url(df)
    i4=has_picture(df)
    return([i1,i2,i3,i4])
########

def distribution_word(df):
    df_mot=df.groupby(df['nb_mots']).count()
    df_mot.to_excel('word_dist_en.xlsx')
    return(df_mot)

## descriptive statistic function

In [3]:
def tweet_by_user(tweet_df) : #give back a data frame with the number of tweet for each user /!\ very heavy !!
    DU=[]
    U=[]
    I=[]
    nb_user=len(tweet_df['fullname'].value_counts())
    df=tweet_df['fullname'].value_counts().sort_values(ascending=False)
    for i in range(nb_user):
        DU.append(df.iloc[i])
        U.append(df.index[i])
        I.append(i)
    d = {'fullname': U, 'nb_tweet': DU}
    df_user = pd.DataFrame(data=d)
    ####
    fig1 = plt.figure(figsize = (30, 10))
    ax = fig1.add_subplot(111)
    
    x = I
    height = DU
    width = 0.5
    plt.tight_layout() #ajustement des etiquttes
    
     #fisrt graph
    plt.bar(x, height, width, color='b')
    
    ax.set_xticklabels(U)
    plt.ylim(0,300)
    plt.xlim(0,100)
    plt.title('user repartition')
    plt.show()
    return(df_user)

def basic_statistic(tweet_df):
    
    # Number of tweets
    nb_tweet=len(tweet_df.index)
    
    print('the campaign is composed of',nb_tweet,'tweets')
    print('\n')
    
    # Number of users involved in the campaign
    nb_users=len(tweet_df['fullname'].value_counts())
    
    print('This Tweets are coming from ',nb_users,'different users')
    print('\n')
    # top users
    print('The top-10 users are :','\n')
    for i in range(10):
        print(tweet_df['fullname'].iloc[i], ' with ', tweet_df['fullname'].value_counts().iloc[i],'tweets')
    print('\n')
    # tweet:
    mean_nb_tweet=round(tweet_df['fullname'].value_counts().mean(),2)
    std_nb_tweet=round(tweet_df['fullname'].value_counts().std(),2)
    med_nb_tweet=round(tweet_df['fullname'].value_counts().median(),2)
    
    print('The average number of tweet by user is :',mean_nb_tweet)
    print('The standard deviation of the number of tweets by user is :',std_nb_tweet)
    print('The  median of the number of tweets by user is :',)
    print('\n')
    
    # Retweets
    nb_retweet=sum(tweet_df['retweets'])/nb_tweet*100
    nb_tweet_retweeted=len(tweet_df[tweet_df['retweets']>1]) /nb_tweet*100
    med_retweet=round(tweet_df['retweets'].mean(),2)
    
    print('The number of retweets involved in this campaign is',nb_retweet)
    print('Among the',nb_tweet,'tweets involved in the campaign,',nb_tweet_retweeted,'were retweeted at least one time')
    print('Each tweets was retweeted on average(Mean) of:',  med_retweet,'times', '\n')
    
    print('\n')
    tweet_df = tweet_df.sort_values(by='retweets', ascending=False)
    tweet_df = tweet_df.reset_index(drop=True)
   
    print('Top 5 Retweeted tweets:')
    print('------------------')
    for i in range(5):
        print(tweet_df['text'].iloc[i], '-', tweet_df['retweets'].iloc[i],'retweets')
    print('\n')
    # Likes
    nb_like=sum(tweet_df['likes'])/nb_tweet*100
    nb_tweet_liked=len(tweet_df[tweet_df['likes']>1])/nb_tweet*100
    mean_like=round(tweet_df['likes'].mean(),2)
    print('The number of likes involved in this campaign is',nb_like)
    print('Among the',nb_tweet,'tweets involved in the campaign,',nb_tweet_liked,'were liked at least one time')
    print('\n')
    tweet_df = tweet_df.sort_values(by='likes', ascending=False)
    tweet_df = tweet_df.reset_index(drop=True)
    print('Each tweets was liked on average(Mean) of:',mean_like ,'times', '\n')
    print('Top 5 liked tweets:')
    print('-------------------')
    for i in range(5):
        print(tweet_df['text'].iloc[i], '-', tweet_df['likes'].iloc[i],'likes')
    print('\n')

    I=['nb_tweet','nb_users','mean_nb_tweet','std_nb_tweet','med_nb_tweet','nb_retweet','nb_tweet_retweeted','nb_like','nb_tweet_liked','mean_like']
    BS=[nb_tweet,nb_users,mean_nb_tweet,std_nb_tweet,med_nb_tweet,nb_retweet,nb_tweet_retweeted,nb_like,nb_tweet_liked,mean_like]
    
    return(BS,I)

def compare_stat(S1,name1,S2,name2,I):
    d = {name1 : S1, name2 : S2}
    df= pd.DataFrame(index=I,data=d)
    df.to_excel('compare_stat.xlsx')
    return(df)

####################### plot time distribution
def time_distribution(tweet_df):
    pd.to_datetime(tweet_df['timestamp'])
    #Create a graph representing the activity of the campaign by day:
    df_D=tweet_df.set_index('timestamp').groupby(pd.TimeGrouper('D')).count().dropna()
    df_H=tweet_df.set_index('timestamp').groupby(pd.TimeGrouper('H')).count().dropna()
    
    times = pd.to_datetime(tweet_df['timestamp'])
    #print(times.dt.hour)
    df_H24=tweet_df.reset_index(drop=True).groupby([times.dt.hour]).count().dropna()
    ################ save and print in excel
    writer = pd.ExcelWriter('time_distribution_fr.xlsx')
    df_D.to_excel(writer,'day_distribution')
    df_H.to_excel(writer,'hour_distribution')
    df_H24.to_excel(writer,'h24_distribution')
    writer.save()
    ############## print in python
    
    dataD=df_D['text']
    dataH=df_H['text']
    dataH24=df_H24['text']
    
    NTD=[]
    NTH=[]
    NTH24=[]
    I=[]
    
    SNTH=[]

    for i in range(len(dataD)):
        NTD.append(dataD[i])
    for i in range(len(dataH)):
        NTH.append(dataH[i])
    
    for i in range(len(dataH24)):
        NTH24.append(dataH24.iloc[i])
        I.append(i)
    
    print(NTH24[:10]) 
    
    SNTH=[NTH[0]]
    for i in range(len(NTH)-1):
        SNTH.append(SNTH[i]+NTH[i+1])
    
#print(TD,data,NTD)
#print(tweet_df['timestamp'].groupby(tweet_df['timestamp'].dayofweek).count())
#tweet_df['hod'] = [t.hour for t in tweet_df['timestamp']]
#tweet_df['dow'] = [t.dayofweek for t in tweet_df['timestamp']]
#print(tweet_df.groupby(tweet_df['timestamp'].map(lambda x: x.day)).count())
#df.set_index('timestamp').groupby(pd.TimeGrouper('D')).count().dropna()['text']

    ###### figure 3
    fig3 = plt.figure(figsize = (20, 10))
    #date = data.index.date.astype('O')
    x = dataD.index.tolist()
    height = NTD
    width = 0.8
    plt.tight_layout() #ajustement des etiquttes
    plt.bar(x, height, width, color='b')
    plt.ylim(0,40000)
    plt.ylabel('number of tweets')
    plt.title('Distribution of tweets by day !')
    plt.show()
    
    #####figure 4
    fig4 = plt.figure(figsize = (20, 10))
    
    #date = data.index.date.astype('O')
    x = dataH.index.tolist()
    height = NTH
    width = 0.01
    plt.tight_layout() #ajustement des etiquttes
    
    plt.bar(x, height, width, color='g')
    plt.ylim(0,2000)
    
    plt.ylabel('number of tweets')
    plt.title('Distribution of tweets  by hour !')
    plt.show()
    print()
     #date = data.index.date.astype('O')
    #####figure 5  
    fig5 = plt.figure(figsize = (20, 10))
    #xlayout=[]
    #for i in range(len(dataH24.index.tolist())) :
    #    t=dataH24.index.tolist()[i]
    #    xlayout.append(str(t[0])+'h'+str(t[1])+'min')
    ##print(x)
    x= dataH24.index.tolist()
    #np.array(range(1,len(dataH24)))
    height = NTH24
    width = 0.8
    plt.tight_layout() #ajustement des etiquttes
    
    plt.bar(x, height, width, color='g')
    plt.ylim(0,10000)
    
    plt.ylabel('number of tweets')
    plt.title('Distribution of tweets  for each hour !')
    plt.show()
    
    #####figure 6
    fig6 = plt.figure(figsize = (20, 10))
    
    #date = data.index.date.astype('O')
    x = dataH.index.tolist()
    height = SNTH
    width = 0.01
    plt.tight_layout() #ajustement des etiquttes
    
    plt.bar(x, height, width, color='g')
    plt.ylim(0,150000)
    
    plt.ylabel('number of tweets')
    plt.title('Cumulative distribution of tweets  by hour !')
    plt.show()

## Hastag anf mentions functions

In [4]:
def list_of_hashtag(tweet_df):
    tweet_df['hashtag'] = tweet_df['hashtag'].str.replace('#', ' ')
    tweet_df['hashtag'] = tweet_df['hashtag'].str.lower()
    remove_ponctuation = lambda x: ''.join(ch for ch in x if ch not in set(string.punctuation))
    tweet_df['hashtag']=tweet_df['hashtag'].map(remove_ponctuation)
    #nltk.word_tokenize(tweet_df['hashtag'][28])
    #create a tab with all the # for frequence analysis
    hashtag=[]
    for text in tweet_df['hashtag']:
        T=nltk.word_tokenize(text)
        
        for j in range(len(T)):
            #print(T[j].lower())
            hashtag.append(T[j].lower())
    return (hashtag)

def most_common_hastag(tab):
    x=Counter(tab)
    H=x.most_common()
    h=[]
    b=[]
    for i in range(100):
        #print(h)
        h.append(H[i][1])
        b.append(H[i][0])
    data={'hashtag':h}
    df=pd.DataFrame(index=b,data=data)
    df.to_excel('mention_frequency_fr.xlsx')
    fig_h = plt.figure(figsize = (20, 7))
    #date = data.index.date.astype('O')
    print(H[:250])
    print(sum(h))
    print(len(tab))
    
    x = range(len(b[:100]))
    height = h[:100]
    labels=b
    width = 0.2
    plt.tight_layout() #ajustement des etiquttes
    plt.bar(x, height, width, color='b')
    plt.xticks(x, labels, rotation='vertical')
    plt.ylabel('hastag frequency')
    plt.title('hastag frequency!')
    plt.show()
    
#####################
#  We create a function that enable to count the number of occurance of a hastag in a serie given

def counthashtag(serie,hashtagR):
    find_hashtag=lambda x: x.find(hashtagR)
    c=0
    for k in range(len(serie)):
        text=serie.iloc[k].lower()
        if find_hashtag(text)>=1:
            c+=1
    return(c)

#######################
def mention_tab(tweet_df):######## Mentions
    remove_ponctuation = lambda x: ''.join(ch for ch in x if ch not in set(string.punctuation))
    tweet_df['mention']=tweet_df['mention'].str.replace('@', ' ')
    tweet_df['mention']=tweet_df['mention'].map(remove_ponctuation)

    mention=[]
    users=0
    for i in range(len(tweet_df['mention'])):
        if len(tweet_df['mention'].iloc[i])>0:
            users+=1
            g=nltk.word_tokenize(tweet_df['mention'].iloc[i])
            for j in range(len(g)):
                mention.append(g[j].lower())
    return(mention)

# terms analisys function

In [5]:
def list_of_word(df_tweet,langue,filename):    
    tweet_mot=[]
    #stop=stopwords.words(langue)+["""’"""]+['…']+['a']
    sw = stop_words.get_stop_words(langue)+["""’"""]+['…']+['a']+['les']+['ça']+['cest']+['jai']+['â€¦']
    file = open(filename,'w',encoding='utf-8')
    for i in range(len(df_tweet['text'])):
        text=nltk.word_tokenize(df_tweet['text'].iloc[i])
        for j in range(len(text)):
            text[j]=text[j].lower()#everithing in lower case
            if text[j] not in sw:
                tweet_mot.append(text[j])
                file.write(text[j])
                file.write('\n')
                
    file.close()
    ### tweet_mot is a list of all the words in the tweets sans les mot dans les stop word
    ####
    return(tweet_mot)

def most_common_word(word_list,k,filename) :
    #unigram
    U=[]
    W=[]
    freq=Counter(word_list)
    uni=freq.most_common(k)
    for i in range(k):
        #print(h)
        U.append(uni[i][1])
        W.append(uni[i][0])
    data={'freq':U,'term':W}
    df_U=pd.DataFrame(data=data)
    print(uni)
    #bigram
    B=[]
    WB=[]
    tweet_bigram = bigrams(word_list)
    freq_bigram= Counter(tweet_bigram)
    bi=freq_bigram.most_common(k)
    
    for i in range(k):
        #print(h)
        B.append(bi[i][1])
        WB.append(str(bi[i][0][0])+' '+str(bi[i][0][1]))
    dataB={'term':WB,'freq':B}
    df_B=pd.DataFrame(data=dataB)
    dict_b = dict(zip(WB, B))
    
    print(bi)
    
    #trigram
    tweet_trigram = trigrams(word_list)
    freq_trigram= Counter(tweet_trigram)
    tri=freq_trigram.most_common(k)
    
    T=[]
    WT=[]
    for i in range(k):
        #print(h)
        T.append(tri[i][1])
        WT.append(str(tri[i][0][0])+' '+str(tri[i][0][1])+' '+str(tri[i][0][2]))
    dataT={'term':WT,'freq':T}
    df_T=pd.DataFrame(data=dataT)
    dict_t = dict(zip(WT, T))
    print(tri)
    
    
    
    writer = pd.ExcelWriter(filename)
    df_U.to_excel(writer,'unigram')
    df_B.to_excel(writer,'bigram')
    df_T.to_excel(writer,'trigram')
    writer.save()
    #return(dict(freq),dict_b,dict_t) #use this to plot wordcloud
    return(df_U,df_B,df_T)#return this one for return dataframe for the sentiment analisys
###################################################################################################
from wordcloud import WordCloud
def plot_wc(filename,filesave):
    text = open(filename,encoding="utf8").read()
    
    # Generate a word cloud image
    wordcloud = WordCloud().generate(text)
    
    # Display the generated image:
    # the matplotlib way:
    
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    
    # lower max_font_size
    wordcloud = WordCloud(max_font_size=40).generate(text)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(filesave)
    plt.show()

def plot_wc_freq(filesave,freq):
    #text = open(filename,encoding="utf8").read()
    
    # Generate a word cloud image
    wordcloud = WordCloud().fit_words(frequencies=freq)
    
    # Display the generated image:
    # the matplotlib way:
    
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    
    # lower max_font_size
    wordcloud = WordCloud(max_font_size=40).fit_words(frequencies=freq)
    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(filesave)
    plt.show()
#######################################

def co_matrix(df_tweet,langue):
    com = defaultdict(lambda : defaultdict(int))
    sw = stop_words.get_stop_words(langue)+["""’"""]+['…']+['a']+['les']+['ça']+['cest']+['jai']+['â€¦']
    # extracting each tweet
    for i in range(len(df_tweet['text'])):
        text=nltk.word_tokenize(df_tweet['text'].iloc[i])
        terms_only = [term for term in text if term not in sw]
     
        # Build co-occurrence matrix
        for k in range(len(terms_only)-1):            
            for j in range(k+1, len(terms_only)):
                w1, w2 = sorted([terms_only[k], terms_only[j]])                
                if w1 != w2:
                    com[w1][w2] += 1
    com_max = []
    # For each term, look for the most common co-occurrent terms
    for t1 in com:
        t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
        for t2, t2_count in t1_max_terms:
            com_max.append(((t1, t2), t2_count))
    # Get the most frequent co-occurrences
    terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
    ##############
    df_com=pd.DataFrame()
    df_com["term"] = np.nan
    df_com["freq"] = np.nan
    for i in range(150):
        df_com.loc[i]=pd.Series({"term":str(terms_max[i][0][0])+' '+str(terms_max[i][0][1]),"freq":terms_max[i][1] })
        
    #df_com.to_excel('com_term_en.xlsx')
    print(terms_max[:20])
    return(terms_max,df_com)

#####################################
def score(text,lang):
    labMT,labMTvector,labMTwordList = emotionFileReader(stopval=0.0,lang=lang,returnVector=True)
    words = text.split()
    s=0
    k=0
    for word in words :
        if word in labMTwordList:
            s+= float(labMT[word][1])
            k+=1 #number of word that labMT count
    if k==0:
        return(0)
    return(s/k)
#####################################
def sentiment_evaluation(df,lang): # THe entry is a data frame composed of 2 column : 'term' and 'freq'  
    #df['score']=np.nan
    S=[]
    for i in range(len(df)):
        S.append(score(df['term'].iloc[i],lang))
    df['score']=S
    df_final=df[df['score']>=1]# we drop all the 0
    df_final['med']=df['score'].median()
    df_final['mean']=df['score'].mean()
    df_final['std']=df['score'].std()
    df_final['freq /1000']=df['freq']/df['freq'].sum()*1000
    df_final.to_excel('score_tab_tri_fr.xlsx')
    return(df_final)


# Other functions

In [6]:
#################### create n  text file with one tweet eact time
def tweet_to_text(df,folder):# de la forme '\folder_name
    newpath = r'C:\Users\Robin'+folder
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    for i in range(len(df['text'])):
        
        filename=folder[1:]+'/tweet'+str(i)+"_"+str(df['id'].iloc[i])+'.txt'
        file = open(filename,'w',encoding='utf-8')
        #for j in range(len(doc_stories_clean[i])):
        text=df['text'].iloc[i]
        file.write(text)
        file.write('\n')
        file.close()
##################"

def create_df_user(df):
    df1=df.groupby(df['users'])
    N=[]
    I=[]
    gb=df1.groups
    for i,item in enumerate(gb) :
        N.append(item)
        I.append(i)
    d = {'id_str': N}
    df_user=pd.DataFrame(index=I,data=d)
    df_user["user_id"] = np.nan
    df_user["screen_name"]= np.nan
    df_user["name"]= np.nan
    df_user["user_description"] = np.nan
    df_user["statuses_count"] = np.nan
    df_user["followers_count"] = np.nan
    df_user["friends_count"] = np.nan
    df_user["langue"] = np.nan
    df_user["created_at"] = np.nan
    
    return(df_user)
############################ create the datframe for each user of the campaign with key information
def user_info(df_user):
    e=0
    k=0
    start_time = time.time()
    print(start_time)
    limit=api.rate_limit_status('users').get('resources').get('users').get('/users/show/:id').get('remaining')
    limit2=api2.rate_limit_status('users').get('resources').get('users').get('/users/show/:id').get('remaining')
    for i in range(len(df_user)):
        if limit>1 :
            try:
                user_info=api.get_user(screen_name=df_user['id_str'].iloc[i])
                
                k+=1
                limit-=1
                print('limit1=',limit)
                df_user.loc[i] = pd.Series({"id_str":df_user['id_str'].iloc[i],"user_id":user_info.id, 
                                            "screen_name":user_info.screen_name, "name":user_info.name,"user_description":user_info.description,
                                            "statuses_count":user_info.statuses_count,"followers_count":user_info.followers_count,
                                            "friends_count":user_info.friends_count, "langue":user_info.lang,
                                            "created_at": user_info.created_at})
            
        
            except:
                e+=1
                limit-=1
                print(e,'erreur pour',k,'succes')
        
        elif limit2>1 :
            print("--- %s seconds ---" % (time.time() - start_time))    
            try:
                user_info=api2.get_user(screen_name=df_user['id_str'].iloc[i])
                k+=1
                limit2-=1
                print('limit2=',limit2)
                df_user.loc[i] = pd.Series({"id_str":df_user['id_str'].iloc[i],"user_id":user_info.id, 
                                            "screen_name":user_info.screen_name, "name":user_info.name,"user_description":user_info.description,
                                            "statuses_count":user_info.statuses_count,"followers_count":user_info.followers_count,
                                            "friends_count":user_info.friends_count, "langue":user_info.lang,
                                            "created_at": user_info.created_at})
            
        
            except:
                e+=1
                limit2-=1    
                print(e,'erreur pour',k,'succes')
        
        else:
            print("--- %s seconds ---" % (time.time() - start_time))
            sleep(15*61-400)
            limit=api.rate_limit_status('users').get('resources').get('users').get('/users/show/:id').get('remaining')
    return(df_user)

# Second cleaning fonctions

In [7]:
def add_nombre_mot(df):
    nb_mots=[]
    for i in range(len(df['text'])):
        text=nltk.word_tokenize(df['text'].iloc[i])
        nb_mots.append(len(text))
    df['nb_mots']=nb_mots
#####################################################################################################################
def distribution(df,column,titre,xlim,ylim): #trace la distribution du colone de dataframe
    N=[]
    I=[]
    for i in range(xlim):
        I.append(i)
        N.append(len(df[df[column]==i]))
    fig1 = plt.figure(figsize = (30, 10))
    x = I
    height = N
    width = 0.5
    plt.tight_layout() #ajustement des etiquttes
    
     #fisrt graph
    plt.bar(x, height, width, color='b')
    plt.ylim(0,ylim)
    plt.xlim(0,xlim)
    plt.title(titre)
    plt.show()

    #############################
    
def drop_by_nb_of_word(df,nb_mots):
    d=0
    e=0
    i=0
    while i <len((df['text'])):
        if df['nb_mots'].iloc[i]<nb_mots:
            try:
                df.drop(df.index[i],inplace=True)
                d+=1
            except:
                e+=1
        i+=1
    print(d,"tweets have been deleted by nb of word", '\n', 'there are',e,'errors')
    return(d)
#############################

def drop_by_user(df,users):
    d=0
    e=0
    i=0
    while i <len((df['user'])):
        if df['user'].iloc[i] in users:
            try:
                df.drop(df.index[i],inplace=True)
                d+=1
            except:
                e+=1
        i+=1
    print(d,"tweets have been deleted by users", '\n', 'there are',e,'errors')
    return(d)
#############################

remove_ponctuation = lambda x: ''.join(ch for ch in x if ch not in set(string.punctuation))
lower= lambda x: x.lower()
def formalize(df):
    df['user']=df['user'].map(lower)
    df['mention']=df['mention'].map(lower)
    df['text']=df['text'].map(lower)
    df['hashtag']=df['hashtag'].map(lower)
    df['mention']=df['mention'].str.replace('@', '')
    df['mention']=df['mention'].map(remove_ponctuation)
#############################

def drop_by_mention(df,mention):
    d=0
    e=0
    i=0
    while i <len((df['text'])):
        if str(df['mention'].iloc[i]) in mention :
            try:
                df.drop(df.index[i],inplace=True)
                d+=1
            except:
                e+=1
        i+=1
    print(d,"tweets have been deleted by mention", '\n', 'there are',e,'errors')
    return(d)
#############################    
def drop_by_hashtag(df,find_hashtag):
    d=0
    e=0
    i=0
    while i <len((df['text'])):
        text=str(df['hashtag'].iloc[i])
        if len(find_hashtag.findall(text))>=int(1):
            try:
                df.drop(df.index[i],inplace=True)
                d+=1
            except:
                e+=1
        i+=1
    print(d,"tweets have been deleted by hashtag", '\n', 'there are',e,'errors')
    return(d)
#############################
def drop_by_word(df,find_word):
    d=0
    e=0
    i=0
    while i <len((df['text'])):
        text=lower(df['text'].iloc[i])
        if len(find_word.findall(text))>=int(1):
            try:
                df.drop(df.index[i],inplace=True)
                d+=1
            except:
                e+=1
        i+=1
    print(d,"tweets have been deleted by word", '\n', 'there are',e,'errors')
    return(d)
#############################

def save(df,filename):
    df.to_json(filename)
    #df2=pd.read_json(filename, orient="columns")

#############################
def cleaning2(df,users,mentions,find_hashtag,find_word):
    df= df.reset_index(drop=True)#réinitialise the index
    add_nombre_mot(df)
    nb_mots=list(df['nb_mots'].quantile([0.1]))[0]
    print(distribution(df,'nb_mots','number of word',75,4000))
    df_final= df[df['nb_mots']>=5]
    print(len(df)-len(df_final),'tweets have been deleted by nb of word')
    print(distribution(df_final,'nb_mots','number of word',75,4000))
    formalize(df_final)
    drop_by_user(df_final,users)
    print(distribution(df_final,'nb_mots','number of word',75,4000))
    drop_by_mention(df_final,mentions)
    print(distribution(df_final,'nb_mots','number of word',75,4000))
    drop_by_hashtag(df_final,find_hashtag)
    print(distribution(df_final,'nb_mots','number of word',75,4000))
    drop_by_word(df_final,find_word)
    print(distribution(df_final,'nb_mots','number of word',75,4000))
       
    return(df_final)    
###############################
def category(df,find_Sexual_assault,find_rape,find_Sexual_harrasment):
    d=0
    e=0
    i=0
    category=[]
    while i <len((df['text'])):
        text=str(df['text'].iloc[i])
        if len(find_Sexual_harrasment.findall(text))>=int(1):
            try:
                category.append('harrasment')
                d+=1
            except:
                category.append(0)
                e+=1
        elif  len(find_Sexual_assault.findall(text))>=int(1):
            try:
                category.append('assault')
                d+=1
            except:
                category.append(0)
                e+=1
        
        elif len(find_rape.findall(text))>=int(1):
            try:
                category.append('rape')
                d+=1
            except:
                category.append(0)
                e+=1
        else :
            category.append(0)
        i+=1
    df['category']=category
    df_harrasment=df[df['category']=='harrasment']
    df_assault=df[df['category']=='assault']
    df_rape=df[df['category']=='rape']
    print('The category harrasment', 'is now composed of :',len(df_harrasment['text']),"tweets", '\n',
          'The category assault', 'is now composed of :',len(df_assault['text']),"tweets", '\n'
          'The category rape', 'is now composed of :',len(df_rape['text']),"tweets", '\n'
          'there are',e,'errors')
    return(df_rape,df_assault,df_harrasment)